# Mask Detection - Training

### Imports

In [9]:
import numpy as np
import glob
import cv2 as cv
import tensorflow as tf
import pickle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

### Constants

In [2]:
classes = {
    "without_mask": 0,
    "with_mask": 1,
    "mask_worn_incorrect": 2
}

one_hot_encoding = {
    0: [1, 0, 0],
    1: [0, 1, 0],
    2: [0, 0, 1]
}

### Utilities

In [3]:
def load_dataset(path, label, labels = classes):
    paths = glob.glob("./" + path + "/"  + label + "/*")
    paths.sort()
    return np.array([np.array(cv.imread(p)) for p in paths]), np.array([labels[label] for p in paths])

def load_resized_dataset(path, label, labels = classes):
    paths = glob.glob("./" + path + "/"  + label + "/*")
    paths.sort()
    return np.array([cv.resize(cv.imread(p), (224, 224), interpolation=cv.INTER_CUBIC) for p in paths]), np.array([labels[label] for p in paths])

def load_double_resized_dataset(path, label, labels = classes):
    paths = glob.glob("./" + path + "/" + label + "/*")
    paths.sort()
    return np.array([np.array(double_resize(cv.imread(p))) for p in paths]), [labels[label] for p in paths]

def load_gray_dataset(label, labels = classes):
    paths = glob.glob("./face-mask-dataset/"  + label + "/*")
    paths.sort()
    return [np.array(cv.cvtColor(cv.imread(p), cv.COLOR_BGR2GRAY)) for p in paths], [labels[label] for p in paths]

def load_images_rgb(path):
    paths = glob.glob(path + "/*")
    paths.sort()
    return [np.array(cv.imread(p)) for p in paths]

def load_images(path):
    paths = glob.glob(path + "/*")
    paths.sort()
    return [np.array(cv.cvtColor(cv.imread(p), cv.COLOR_BGR2GRAY)) for p in paths]

def extract_hog(image, ppc = 16):
    features, _ = hog(image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
    return features

def double_resize(image, first_size = (30, 30), target_size = (224, 224)):
    resized = cv.resize(image, (first_size), interpolation = cv.INTER_CUBIC)
    return cv.resize(resized, (target_size), interpolation = cv.INTER_CUBIC)

## SVM Classifier

In [11]:
mask, mask_labels = load_gray_dataset("with_mask")
no_mask, no_mask_labels = load_gray_dataset("without_mask")
mask_incorrect, mask_incorrect_labels = load_gray_dataset("mask_worn_incorrect")

print(len(mask), len(mask_labels))
print(len(no_mask), len(no_mask_labels))
print(len(mask_incorrect), len(mask_incorrect_labels))

images = np.concatenate((mask, no_mask, mask_incorrect))
labels = np.concatenate((mask_labels, no_mask_labels, mask_incorrect_labels))

images = np.concatenate((images, [cv.flip(i, 1) for i in images]))
labels = np.concatenate((labels, labels))

print(len(images), len(labels))

for i in images:
    assert(i.shape == (128, 128))

2994 2994
2994 2994
2994 2994
17964 17964


In [12]:
hog_train = [extract_hog(image) for image in images]

In [6]:
model = svm.SVC()
parameters = [{
    "C": [0.1, 1, 10],
    "kernel": ["poly"],
    "degree": [2, 3, 4, 5],
    "gamma": [0.05, 0.025]
}, {
    "C": [0.1, 1, 10],
    "kernel": ["rbf"],
    "gamma": [0.05, 0.025]
}]
n_folds = 3
grid_search_cv = GridSearchCV(model, parameters, cv=n_folds)

grid_search_cv.fit(hog_train, labels)
print(grid_search_cv.best_params_)

{'C': 10, 'degree': 3, 'gamma': 0.05, 'kernel': 'poly'}


In [13]:
clf = svm.SVC(C=10, degree=3, gamma=0.05, kernel="poly")
clf.fit(hog_train, labels)

SVC(C=10, gamma=0.05, kernel='poly')

In [14]:
filename = "svm-model2.pkl"
with open(filename, 'wb') as file:
    pickle.dump(clf, file)

In [5]:
filename = "svm-model.pkl"
with open(filename, 'rb') as file:
    clf = pickle.load(file)

In [144]:
faces = load_images_rgb("faces")
resized_faces = [cv.resize(f, (128, 128), interpolation=cv.INTER_CUBIC) for f in faces]

In [15]:
hog_faces = [extract_hog(f) for f in resized_faces]
print(clf.score(hog_faces,  [1 for i in range(len(faces))]))
results = clf.predict(hog_faces)

In [30]:
print([e for e in zip([i for i, c in enumerate(results) if c != 1],
    [c for c in results if c != 1])])

[(10, 0), (15, 0), (18, 0), (21, 2), (25, 0), (28, 0), (48, 0), (51, 0), (124, 0), (127, 0), (131, 0)]


## SSD-MobilenetV2 Classifier

In [4]:
input_shape = (224, 224, 3)
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.3),
        layers.RandomZoom(0.2)
    ]
)
inputs = keras.Input(shape=input_shape)
augmentation = data_augmentation(inputs)
mobilenet = tf.keras.applications.mobilenet_v2.MobileNetV2(weights="imagenet", include_top=False, input_shape=input_shape)(augmentation)
maxpool = tf.keras.layers.GlobalMaxPooling2D()(mobilenet)
output = tf.keras.layers.Dense(3, activation='softmax')(maxpool)
model = tf.keras.Model(inputs=[inputs], outputs=[output])

for layer in model.layers[:-23]:
    layer.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])    

2022-03-29 14:21:31.197310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-29 14:21:31.217670: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-29 14:21:31.218180: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-29 14:21:31.224276: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [5]:
double_resized = True

mask, mask_labels = load_double_resized_dataset("face-mask-dataset", "with_mask") if double_resized else load_resized_dataset("face-mask-dataset", "with_mask")
no_mask, no_mask_labels = load_double_resized_dataset("face-mask-dataset", "without_mask") if double_resized else load_resized_dataset("face-mask-dataset", "without_mask")
mask_incorrect, mask_incorrect_labels = load_double_resized_dataset("face-mask-dataset", "mask_worn_incorrect") if double_resized else load_resized_dataset("face-mask-dataset", "mask_worn_incorrect")

In [6]:
print(len(mask), len(mask_labels))
print(len(no_mask), len(no_mask_labels))
print(len(mask_incorrect), len(mask_incorrect_labels))

images = np.concatenate((mask, no_mask, mask_incorrect))
labels = np.concatenate((mask_labels, no_mask_labels, mask_incorrect_labels))
labels = np.array([one_hot_encoding[i] for i in labels])

images = tf.keras.applications.mobilenet_v2.preprocess_input(images)
print(len(images), len(labels))
for i in images:
    assert(i.shape == (224, 224, 3))

2994 2994
2994 2994
2994 2994
8982 8982


In [7]:
split = False

if split:
    x_train, x_test, y_train, y_test = train_test_split(images, labels, train_size=0.8, shuffle=True, random_state=0)
else:
    x_train = images
    y_train = labels

In [8]:
def lr_schedule(epoch):
    if epoch < 5:
        return 0.0001
    if epoch < 15:
        return 0.00005
    return 0.00001

callbacks = [
    tf.keras.callbacks.LearningRateScheduler(schedule=lr_schedule),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='mobilenet_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5'
    )
]

In [ ]:
history = model.fit(x=x_train,
            y=y_train,
            validation_split=0.2,
            epochs=200,
            shuffle=True,
            callbacks=callbacks
)

2022-03-29 14:21:55.649470: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4326174720 exceeds 10% of free system memory.
2022-03-29 14:21:57.377475: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4326174720 exceeds 10% of free system memory.


Epoch 1/200


2022-03-29 14:22:02.643298: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


225/225 [==============================] - 37s 138ms/step - loss: 0.7403 - accuracy: 0.8168 - val_loss: 0.2477 - val_accuracy: 0.9082 - lr: 1.0000e-04
Epoch 2/200
225/225 [==============================] - 29s 130ms/step - loss: 0.3256 - accuracy: 0.9065 - val_loss: 0.1373 - val_accuracy: 0.9449 - lr: 1.0000e-04
Epoch 3/200
225/225 [==============================] - 29s 131ms/step - loss: 0.2552 - accuracy: 0.9297 - val_loss: 0.1146 - val_accuracy: 0.9588 - lr: 1.0000e-04
Epoch 4/200
225/225 [==============================] - 30s 131ms/step - loss: 0.2149 - accuracy: 0.9396 - val_loss: 0.0481 - val_accuracy: 0.9816 - lr: 1.0000e-04
Epoch 5/200
225/225 [==============================] - 30s 132ms/step - loss: 0.1662 - accuracy: 0.9549 - val_loss: 0.4384 - val_accuracy: 0.8436 - lr: 1.0000e-04
Epoch 6/200
225/225 [==============================] - 30s 132ms/step - loss: 0.1325 - accuracy: 0.9609 - val_loss: 0.1459 - val_accuracy: 0.9471 - lr: 5.0000e-05
Epoch 7/200
225/225 [=============

225/225 [==============================] - 30s 133ms/step - loss: 0.0064 - accuracy: 0.9972 - val_loss: 0.0026 - val_accuracy: 0.9994 - lr: 1.0000e-05
Epoch 102/200
225/225 [==============================] - 30s 132ms/step - loss: 0.0037 - accuracy: 0.9987 - val_loss: 0.0031 - val_accuracy: 0.9989 - lr: 1.0000e-05
Epoch 103/200
225/225 [==============================] - 30s 132ms/step - loss: 0.0064 - accuracy: 0.9981 - val_loss: 0.0016 - val_accuracy: 0.9994 - lr: 1.0000e-05
Epoch 104/200
225/225 [==============================] - 30s 132ms/step - loss: 0.0053 - accuracy: 0.9985 - val_loss: 0.0051 - val_accuracy: 0.9983 - lr: 1.0000e-05
Epoch 105/200
225/225 [==============================] - 30s 132ms/step - loss: 0.0033 - accuracy: 0.9987 - val_loss: 0.0026 - val_accuracy: 0.9989 - lr: 1.0000e-05
Epoch 106/200
225/225 [==============================] - 30s 132ms/step - loss: 0.0053 - accuracy: 0.9981 - val_loss: 0.0011 - val_accuracy: 0.9994 - lr: 1.0000e-05
Epoch 107/200
225/225 [=